In [ ]:
#import necessary libraries
import asyncio
import requests
import pandas as pd
import websockets
import json
from backtesting import Backtest, Strategy
import talib as ta

# Fetch historical data from Binance API
def fetch_historical_data(symbol, interval, limit=1000):
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}'
    data = requests.get(url).json()
    lst = [i[:7] for i in data]
    df = pd.DataFrame(lst, columns=['StartTime', 'Open', 'High', 'Low', 'Close', 'Volume', 'EndTime'])
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    return df

# WebSocket handler to fetch live data
async def fetch_live_data(symbol, interval, df):
    url = f'wss://stream.binance.com:9443/ws/{symbol.lower()}@kline_{interval}'
    async with websockets.connect(url) as wb:
        while True:
            message = await wb.recv()
            m = json.loads(message)
            kline = m['k']
            data = [{
                "StartTime": kline['t'],
                "Open": float(kline['o']),
                "High": float(kline['h']),
                "Low": float(kline['l']),
                "Close": float(kline['c']),
                "Volume": float(kline['v']),
                "EndTime": kline['T'] }]

            df1 = pd.DataFrame(data)
            if df.iloc[-1]['StartTime'] == df1.iloc[0]['StartTime']:
                df.iloc[-1] = df1.iloc[0]
            else:
                df = pd.concat([df, df1], ignore_index=True)

            # Automatically run backtesting after updating DataFrame
            run_backtest(df)

# Strategy using Bollinger Bands
class BollingerBands(Strategy):
    def init(self):
        self.upper, self.middle, self.lower = self.I(ta.BBANDS, self.data.Close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

    def next(self):
        if self.data.Close[-1] < self.lower[-1]:
            self.buy()
        elif self.data.Close[-1] > self.upper[-1]:
            self.sell()

# Backtesting function
def run_backtest(df):
    bt = Backtest(df,BollingerBands,commission=0.002,exclusive_orders=True,cash=10000000)
    stats = bt.run()
    print(stats)
    bt.plot()

#function to run the program
async def execute():
    symbol = "BTCUSDT"
    interval = "5m"

    # Fetch historical data
    df = fetch_historical_data(symbol, interval)

    # Start live data fetching and automate backtesting
    await fetch_live_data(symbol, interval, df)

# Start the program directly
asyncio.run(execute())
